In [1]:
### Utility Functions
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [38]:
def create_covid19_table(data_filename, normalized_database_filename):

    conn = create_connection('Covid19.db')

    
    create_table_covid19_sql = """ CREATE TABLE [Covid19] (
    [submission_date] DATE NOT NULL,
    [state] TEXT NOT NULL,
    [tot_cases] INTEGER,
    [conf_cases] INTEGER,
    [prob_cases] INTEGER,
    [new_case] INTEGER,
    [pnew_case] INTEGER,
    [tot_death] INTEGER, 
    [conf_death] INTEGER,
    [prob_death] INTEGER,
    [new_death] INTEGER,
    [pnew_death] INTEGER,
    [created_at] TEXT NOT NULL,
    [consent_cases] TEXT NOT NULL,
    [consent_deaths] TEXT NOT NULL
    ); """
    
    def insert_covid19(conn, values):
        insert_covid_19_sql = ''' INSERT INTO Covid19(submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths) 
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?) '''
        cur = conn.cursor()
        cur.executemany(insert_covid_19_sql, values)
        return cur.lastrowid        

    with open(data_filename, 'r') as f:
        covid19_lst = []
        for line in f:
            covid19_lst.append(line.rstrip('\n').split('\t'))
        covid19_lst=sorted(covid19_lst[1:])
        #print(covid19_lst[1:10])
        
    with conn:
        lst_tup_reg=[]
        lst=[]
        create_table(conn, create_table_covid19_sql,'Covid19')
        for i in range(len(covid19_lst)):
            lst_tup_reg.append(tuple(''.join(covid19_lst[i]).split(',')))
        #print(lst_tup_reg[1:10])
        insert_covid19(conn,lst_tup_reg)

In [39]:
data_filename = 'United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv'
normalized_database_filename = 'Covid19.db'
create_covid19_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM Covid19""", conn)
print(df)
conn.close()

      submission_date state  tot_cases conf_cases prob_cases new_case  \
0          01/01/2021    AK      46241                            470   
1          01/01/2021    AL     365720     298102      67618     3626   
2          01/01/2021    AR     229442                           4304   
3          01/01/2021    AS          3                              0   
4          01/01/2021    AZ     530622     504153      26469    10063   
...               ...   ...        ...        ...        ...      ...   
41635      12/31/2020    VT       6966                            138   
41636      12/31/2020    WA     246413                           3958   
41637      12/31/2020    WI     520438     481102      39336     4212   
41638      12/31/2020    WV      85334      69506      15828     1109   
41639      12/31/2020    WY      44409      38010       6399      276   

      pnew_case  tot_death conf_death prob_death new_death pnew_death  \
0             0        285                        